In [10]:
def B1(thickness, TVL): # Calculates B factor using only 1 TVL. Important for PATIENT SCATTER
    '''
    param thickness: thickness of barrier (cm)
    param TVL: TVL (cm) where it is ALWAYS dependent on energy. Can also depend on the angle too
    '''
    B = 10**-(thickness/TVL)
    return B

def B2(thickness, TVL1, TVLe): # Calculates B factor using 2 TVLs. Important for LEAKAGE
    '''
    param thickness: thickness of barrier (cm)
    param TVL1: first TVL (cm)
    param TVLe: equilibrium TVL (cm)
    '''
    B = 10**-(1+((thickness-TVL1)/TVLe))
    return B

def H_ps_unshielded(W, d_secondary, d_scatter, a, F): # Calculates unshielded equivalent dose for PATIENT SCATTER
    '''
    param W: workload in a typical week (Gy/wk)
    param d_secondary: distance from isocenter to the point of interest (cm). Includes the 0.3 m
    param d_scatter: distance from target to isocenter. Usually 1 m
    param a: scatter fraction dependent on angle (deg) and beam energy (MV)
    param F: field size in cm^2
    '''
    H_ps_uns = (W * a * F) / (d_secondary**2 * d_scatter**2 * 400) * 10**3
    return H_ps_uns # (mSv/wk)

def H_leak_unshielded(W_L, d_leakage): # Calculates UNSHIELDED equivalent dose for LEAKAGE
    '''
    param W_L: leakage workload (Gy/wk)
    param d_leakage: distance from source of leakage to point of protection (cm)
    '''
    H_leak_uns = ((W_L*10**-3) / d_leakage**2) * 10**3 # 10**-3 is the included barriers from the head and 10**3 converts Sv to mSv
    return H_leak_uns # (mSv/wk)

def verify_2barrier(P, T, thickness,d,tvl_6mv_ps,tvl_18mv_ps,a_6mv,a_18mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param tvl_18mv_ps: TVL for 18 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    param a_18mv: scatter fraction for 18 MV
    '''
  
    H_l_6mv = H_leak_unshielded(1990, d) * B2(thickness, 34, 29)
    H_l_18mv = H_leak_unshielded(250, d) * B2(thickness, 36, 34)

    H_ps_6mv = H_ps_unshielded(520, d, 1, a_6mv, 1600) * B1(thickness, tvl_6mv_ps)
    H_ps_18mv = H_ps_unshielded(250, d, 1, a_18mv, 1600) * B1(thickness, tvl_18mv_ps)

    H_total = (H_ps_6mv + H_ps_18mv + H_l_6mv + H_l_18mv) * 1e3 # Convert to uSv/wk
    
    if H_total < (P /T) * 1e3: # Convert to uSv/wk
        print("PASS")
    else:
        print("FAIL")
    
    print("H_leak_6mv: ", H_l_6mv, "uSv/wk")
    print("H_leak_18mv: ", H_l_18mv, "uSv/wk")
    print("H_ps_6mv: ", H_ps_6mv, "uSv/wk")
    print("H_ps_18mv: ", H_ps_18mv, "uSv/wk")
    print("----------------------------------")
    print("Total dose: ", H_total, "uSv/wk")
    print("Dose limit: ", (P / T) * 1e3, "uSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"distance from target to 0.3m beyond distal surface: {d} m")
    print(f"TVL_ps (6MV) = {tvl_6mv_ps} cm")
    print(f"TVL_ps (18MV) = {tvl_18mv_ps} cm")
    print(f"Scatter Fraction (6MV) = {a_6mv}")
    print(f"Scatter Fraction (18MV) = {a_18mv}")
    print("--------------------------------------")
    print(f"TVL_1 (6MV) = 34 cm")
    print(f"TVL_e (6MV) = 29 cm")
    print(f"TVL_1 (18MV) = 36 cm")
    print(f"TVL_e (18MV) = 34 cm")
# Important note: thickness must be in centimeters.

def verify_2barrier_vault2(P, T, thickness,d,tvl_6mv_ps,a_6mv):
    '''
    param P: design goal (mSv/wk)
    param T: occupancy factor (unitless)
    param thickness: thickness of barrier (cm)
    param d: distance from target to 0.3m beyond distal surface (cm)
    param tvl_6mv_ps: TVL for 6 MV (cm)
    param a_6mv: scatter fraction for 6 MV 
    '''
  
    H_l_6mv = H_leak_unshielded(1435.5, d) * B2(thickness, 34, 29)

    H_ps_6mv = H_ps_unshielded(319, d, 1, a_6mv, 400) * B1(thickness, tvl_6mv_ps)

    H_total = (H_ps_6mv + H_l_6mv) * 1e3 # Convert to uSv/wk
    if H_total < (P/T) * 1e3: # Convert to uSv/wk
        print("PASS")
    else:
        print("FAIL")
    print(f"H_ps_6mv: {H_ps_6mv} uSv/wk")
    print(f"H_l_6mv: {H_l_6mv} uSv/wk")
    print("Total dose: ", H_total, "uSv/wk")
    print("Dose limit: ", (P / T) * 1e3, "uSv/wk")
    print(f"Thickness used: {thickness} cm = {thickness /2.54} inches")
    print(f"TVL (6MV) = {tvl_6mv_ps} cm")
    print(f"TVL1 (6MV) = 34 cm")
    print(f"TVLe (6MV) = 29 cm")
    print(f"Scatter Fraction (6MV) = {a_6mv}")
    print(f"Workload PS (6MV) = 1435.5 Gy/wk")
    print(f"Workload Leak = 319 Gy/wk")
    print(f"distance = {d} m")


In [11]:
d_iso_to_pt2_vault1 = convert_to_meters(30,9.23883) # m
d_iso_to_pt2_vault2 = convert_to_meters(30,9.19961) # m

slanted_thickness_vault1 = convert_to_centimeters(5,3.04876) # cm
slanted_thickness_vault2 = convert_to_centimeters(4,10.64736) # cm

verify_2barrier(0.02,1,slanted_thickness_vault1,d_iso_to_pt2_vault1,26,32,1.39e-3,8.64e-4) # vault 1
verify_2barrier_vault2(0.02,1,slanted_thickness_vault2,d_iso_to_pt2_vault2,26,1.39e-3) # vault 2

PASS
H_leak_6mv:  0.00010110740546608517 uSv/wk
H_leak_18mv:  6.344072325008687e-05 uSv/wk
H_ps_6mv:  2.2772835452948387e-05 uSv/wk
H_ps_18mv:  9.721566514148666e-05 uSv/wk
----------------------------------
Total dose:  0.2845366293106071 uSv/wk
Dose limit:  20.0 uSv/wk
Thickness used: 160.14385040000002 cm = 63.04876000000001 inches
distance from target to 0.3m beyond distal surface: 9.378666282 m
TVL_ps (6MV) = 26 cm
TVL_ps (18MV) = 32 cm
Scatter Fraction (6MV) = 0.00139
Scatter Fraction (18MV) = 0.000864
--------------------------------------
TVL_1 (6MV) = 34 cm
TVL_e (6MV) = 29 cm
TVL_1 (18MV) = 36 cm
TVL_e (18MV) = 34 cm
PASS
H_ps_6mv: 9.40198695814398e-06 uSv/wk
H_l_6mv: 0.00017722593759371744 uSv/wk
Total dose:  0.18662792455186142 uSv/wk
Dose limit:  20.0 uSv/wk
Thickness used: 148.9642944 cm = 58.64736 inches
TVL (6MV) = 26 cm
TVL1 (6MV) = 34 cm
TVLe (6MV) = 29 cm
Scatter Fraction (6MV) = 0.00139
Workload PS (6MV) = 1435.5 Gy/wk
Workload Leak = 319 Gy/wk
distance = 9.37767009

In [2]:
total_dose_at_point2 = 0.28453 + 0.1866 
print(f"Secondary Radiation at Point 2 from both vaults: {total_dose_at_point2} uSv/wk")

Secondary Radiation at Point 2 from both vaults: 0.47113 uSv/wk
